<a href="https://colab.research.google.com/github/MaxSimonNm/GCloud_Work/blob/main/Sample_QC_Extractor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import auth
import gspread
from google.auth import default
import pandas as pd
from gspread_dataframe import set_with_dataframe
#autenticating to google
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

In [ ]:
# Get the instance of the Spreadsheet
spreadsheet = gc.open('Samples QC')  # replace with your Google Sheet name

# Get the specific sheet name
worksheet = spreadsheet.worksheet('Germline Ref 3.9.5 DQCM Jan 23-Dec 23')  # replace with your sheet name

# Get all values in the sheet
rows = worksheet.get_all_values()

# Convert the data to a pandas DataFrame
df = pd.DataFrame(rows[1:], columns=rows[0])  # Set the first row as header

# Convert numeric columns to appropriate data types
numeric_columns = ['Total aligned bases', 'TAB Score', 'Data received -Total data (GB)',
                   'Total length of targeted reference', 'Percent duplicate aligned reads',
                   'PDAR Score', 'Unique base enrichment', 'UBE Score',
                   'Mean target coverage depth', 'MTCD Score',
                   'Uniformity of coverage (Pct > 0.2*mean)', 'UC Score',
                   'Fragment length median', 'FLM Score', 'Total Score', 'countif']

In [ ]:
# Display the DataFrame
print(df)

     Service provider-Date (LifeCell/Wipro/Medgenome/CDFC)     ID  \
0                                    L-B109(06/02/2023)      WATD   
1                                    L-B109(06/02/2023)      WATJ   
2                                    L-B109(06/02/2023)      WATA   
3                                    L-B109(06/02/2023)      WATL   
4                                    L-B109(06/02/2023)      WATC   
...                                                 ...       ...   
1338                                 W-166 (23/10/2023)     WJNAK   
1339                                 W-166 (23/10/2023)     WJJAA   
1340                                 W-166 (23/10/2023)     WJJAG   
1341                                 W-166 (23/10/2023)     WJMAC   
1342                                 W-166 (23/10/2023)     WJLAK   

                 Sample Name  \
0     WATD-B-SE8-CL-S44-L004   
1         WATJ-B-CE-S45-L004   
2         WATA-B-CE-S42-L004   
3         WATL-B-CE-S47-L004   
4         W

In [ ]:
# List of sample names
sample_names = ['WB2H-B2-SE8-CL', 'WCUF-B-Res-D-SE8-CL-S13', 'WDFF-B1-B2-Res-SE8-CL-S11-L004', 'WE4R-B1-Res-D2-L2-L3-SE8-CL-merged-S39-S40', 'WEJP-B1-B2-Res-SE8-CL', 'WEVQ-B1-Res-D-SE8-CL-S41', 'WEWK-B1-Res-D-SE8-CL-S42', 'WFGT-B1-Res-D-SE8-CL-S267-L004', 'WJ7A6-B-D-L1-SE8-S1', 'WKPAM-B-Res-D-L1L2-SE8-S1', 'WLRAH-B-Res-D1D2-SE8-merged-S219-L004', 'WLSA9-B1B2-Res-D-SE8-S225-L004', 'WLTAC-B-D-L2-SE8-S1', 'XACAH-B-D4-SE8-S158-L004']

# Define the columns you want to extract
columns_of_interest = [
    'Sample Name', 'Target manifest', 'Total aligned bases', 'TAB Score',
    'Percent duplicate aligned reads', 'PDAR Score', 'Unique base enrichment',
    'UBE Score', 'Mean target coverage depth', 'MTCD Score',
    'Uniformity of coverage (Pct > 0.2*mean)', 'UC Score',
    'Fragment length median', 'FLM Score', 'Total Score'
]

# Add missing columns with empty values
for col in columns_of_interest:
    if col not in df.columns:
        df[col] = ''

# Filter the DataFrame for the sample names
filtered_df = df[df['Sample Name'].isin(sample_names)]

# Select the columns of interest
filtered_df = filtered_df[columns_of_interest]

# Find sample names not in the DataFrame
found_sample_names = filtered_df['Sample Name'].tolist()
missing_sample_names = [name for name in sample_names if name not in found_sample_names]

# Display the filtered DataFrame
print("Filtered DataFrame:")
print(filtered_df)

# Display the sample names not found
print("\nSample names not found:")
print(missing_sample_names)

Filtered DataFrame:
                                    Sample Name  \
321                     WCUF-B-Res-D-SE8-CL-S13   
339              WDFF-B1-B2-Res-SE8-CL-S11-L004   
341                              WB2H-B2-SE8-CL   
570                       WEJP-B1-B2-Res-SE8-CL   
618                    WEVQ-B1-Res-D-SE8-CL-S41   
620                    WEWK-B1-Res-D-SE8-CL-S42   
621  WE4R-B1-Res-D2-L2-L3-SE8-CL-merged-S39-S40   
668              WFGT-B1-Res-D-SE8-CL-S267-L004   

                      Target manifest Total aligned bases TAB Score  \
321  SureSelectXT_V8_Covered_used.bed          4763613157             
339  SureSelectXT_V8_Covered_used.bed          5175342290             
341  SureSelectXT_V8_Covered_used.bed          4508746750             
570  SureSelectXT_V8_Covered_used.bed          6659771195             
618  SureSelectXT_V8_Covered_used.bed          6116129368             
620  SureSelectXT_V8_Covered_used.bed          5536902126             
621  SureSelectXT_V8_Co

In [ ]:
# Create a new Google Sheet and write the DataFrame to it
new_spreadsheet = gc.create('GBC_Germline_DNA_QC')  # replace with your desired sheet name
new_worksheet = new_spreadsheet.get_worksheet(0)

# Write the DataFrame to the new worksheet
set_with_dataframe(new_worksheet, filtered_df)

# Share the new spreadsheet (optional)
# new_spreadsheet.share('your_email@example.com', perm_type='user', role='writer')